In [ ]:
#@title # **<font color=gold>1.</font>** Upload files and setup code
Static_Image_URL = "" #@param {type: "string"}
Audio_Top_URL = "" #@param {type: "string"}
Audio_Bottom_URL = "" #@param {type: "string"}
#@markdown ℹ️ Leave empty for manual upload.

#-=-=-=-#

!pip install ffpb mutagen -q

import os
from zipfile import *
from PIL import Image as IMG
from base64 import b64encode
from requests import Session
from shutil import copyfileobj
from google.colab import files
from mutagen import File as mFile
from IPython.display import display, HTML

#-=-=-=-#

!rm -rf *
Error = lambda exc: SystemExit(exc[0].upper() + exc[1:] + ".")

session = Session()
session.headers.update(
	{
		"User-Agent":
		"Mozilla/5.0 (Windows NT 10.0; Win32; x32) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
	}
)
get = session.get

def Clamp(Number: float, Minimum: float, Maximum: float) -> float:
	"""Limits `Number` in range (`Minimum`, `Maximum`)."""
	return max(min(Maximum, Number), Minimum)

def Download(URL: str, Verbose: bool = True) -> str:
	"""Downloads the URL to local directory."""
	URL = URL.split("#")[0].split("?")[0].split("&")[0]
	File_Name = URL.split("/")[-1]
	#-=-=-=-#
	try:
		with get(URL, stream = 1) as Site:
			if Site.ok:
				with open(File_Name, "wb") as File:
					if Verbose:
						print(f"Downloading {File_Name}...")
					copyfileobj(Site.raw, File)
			else:
				raise Error(f"URL returned status code {Site.stats_code} - {Site.reason.title()}")
	except OSError:
		raise Error("Invalid URL")
	#-=-=-=-#
	return File_Name

#-=-=-=-#

Files_Upload_Dict = {
	"Static_Image":  Static_Image_URL,
	"Audio_Top":  Audio_Top_URL,
	"Audio_Bottom":  Audio_Bottom_URL
}

for Key, Value in Files_Upload_Dict.items():
	if "Top" in Key:
		if not Value:
			print()
	#-=-=-=-#
	if Value:
		exec(f'{Key} = Download("{Value}")')
	else:
		print(f'Upload {" ".join(Key.split("_")).upper()}...')
		try:
			exec(f"{Key} = list(files.upload().keys())[-1]")
		except:
			raise Error("File selection aborted")
	exec(f'{Key} = {Key}.split(os.sep)[-1]')

IMG_File = Static_Image
SND_Top = Audio_Top
SND_Bottom = Audio_Bottom

#-=-=-=-#

print("\nUploaded successfully!")

In [ ]:
#@title # **<font color=gold>2.</font>** Encode
#@markdown ## Video 📹
Preview_Width = 250 #@param {type: "slider", min: 75, max: 500, step: 1}
Preview_Width = str(Preview_Width)

#@markdown ---
#@markdown ## Audio 🔊
Top_Gain = -2.5 #@param {type: "slider", min: -5, max: 5, step: 0.1}
Inverse_Positions = False #@param {type: "boolean"}

Top_Gain = str(Top_Gain)
if Inverse_Positions:
	SND1 = SND_Top
	SND2 = SND_Bottom
	#-=-=-=-#
	SND_Top = SND2
	SND_Bottom = SND1
	#-=-=-=-#
	del SND1, SND2

#-=-=-=-#

Video = ".".join(IMG_File.split(os.sep)[-1].replace(" ", "_").split(".")[:-1])

env = os.environ
env["IMG"] = IMG_File

Minimum_Bitrate = 16 * 6
for Key, Value in {"Top":  1, "Bottom":  2}.items():
	SND = "SND_" + Key

	_AF = "AF_" + Key
	_Box = "Box_" + Key
	_Gain = "Gain_" + Key
	_Bitrate = "Bitrate_" + Key
	_Duration = "Duration_" + Key
	_Video = "Video_" + Key

	#-=-=-=-#

	try:
		exec(f'{_AF} = mFile({SND})')
		env[SND] = eval(SND)
	except:
		raise Error("Files not found")
	env[_Bitrate] = str(eval(
		f"Clamp({_AF}.info.bitrate // 1000, Minimum_Bitrate, Minimum_Bitrate * 4)"
		)
	)
	env[_Box] = "iw:ih/2:0:0" if Key == "Top" else "iw:ih/2:0:ih/2"
	env[_Gain] = Top_Gain if Key == "Top" else "0"
	env[_Bitrate] += "k"
	env[_Duration] = str(eval(f"{_AF}.info.length"))
	env[_Video] = Video + f"_{Value}.mp4"
	print(f"Encoding {Key.lower()} video...")

	#-=-=-=-#

	!ffpb -y -strict 2 -loop 1 -ss 0 -i "{IMG_File}" -i "{env[SND]}" \
	-movflags faststart \
	-vf "pad=ceil(iw/2)*2:ceil(ih/2)*2,crop={env[_Box]}" \
	-af "volume={env[_Gain]}dB" \
	-pix_fmt yuv420p -tune stillimage -sws_flags lanczos \
	-c:v libx264 -c:a aac -b:a {env[_Bitrate]} -to {env[_Duration]} "{env[_Video]}"
	
	#-=-=-=-#

	print()

#-=-=-=-#


display(
	HTML("""
		<hr>
		<div style="
			width: 100% !important;
			display: flex !important;
			flex-flow: column !important
		">
			<video controls autoplay width={2} onloadstart="this.volume=.5">
				<source src="{0}" type="video/mp4">
			</video>
			<video controls autoplay width={2} onloadstart="this.volume=.5">
				<source src="{1}" type="video/mp4">
			</video>
		</div>
		<hr>
	""".format(
		"data:video/mp4;base64," + b64encode(open(f"{Video}_1.mp4", "rb").read()).decode(),
		"data:video/mp4;base64," + b64encode(open(f"{Video}_2.mp4", "rb").read()).decode(),
		Preview_Width
		,)
	)
)

#-=-=-=-#

os.chdir("/content/")
print("Downloading archive...")
Archive_Name = f"{Video}_SBS.zip"
Archive = ZipFile(Archive_Name, "w", 14)
for Number in [1, 2]:
	Archive.write(f"{Video}_{str(Number)}.mp4")

files.download(Archive_Name)